In [24]:
from diffusers import StableDiffusionPipeline
import torch

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 2. Model Loading with Optimizations
pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    safety_checker=None,  # Disable if using CPU for speed
    requires_safety_checker=False
).to(device)

Using device: cpu


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 23.50it/s]


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def generate_mochi_image(activity, seed=None):
    prompt = f"""
    A maltese dog doing this activity: {activity.lower()}
    """
    
    generator = torch.Generator(device).manual_seed(seed) if seed else None
    
    image = pipe(
        prompt, 
        num_inference_steps=50,
        generator=generator
    ).images[0]
    
    # Display in notebook
    plt.figure(figsize=(5,5))
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    
    return image

In [5]:
# Example usage
activity = "Sleeping"  # Replace with your model's prediction
image = generate_mochi_image(activity)

# Save to file
image.save("mochi_sleep.jpg")

  0%|          | 0/50 [03:24<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import joblib
import numpy as np
import pandas as pd

# Load your trained model
model = joblib.load('model.pkl')

def preprocess_input(time, day):
    """Convert time and day into model-ready features"""
    try:
        # Convert time to float
        hours, minutes = map(int, time.split(':'))
        time_float = hours + minutes/60
        
        # Create feature DataFrame (MUST match your training data columns)
        features = {
            'hour_sin': np.sin(2 * np.pi * time_float/24),
            'hour_cos': np.cos(2 * np.pi * time_float/24),
            'Day': day,
            'is_weekend': int(day in ['Saturday', 'Sunday']),
            
            # Add defaults for other features your model expects
            'Duration_minutes': 10,          # Example default
            'Location': 'Living room',       # Example default
            'Weather': 'Sunny',              # Example default
            'People_home': 2,                # Example default
            'Mood': 'Neutral',               # Example default
            'Reward_given': 0                # Example default
        }
        
        # Convert to DataFrame with correct column order
        return pd.DataFrame([features])[model.feature_names_in_]
    
    except Exception as e:
        print(f"Preprocessing error: {str(e)}")
        raise


In [ ]:

def predict_and_show(time, day):
    """Predict activity and generate image"""
    try:
        # Preprocess input
        input_data = preprocess_input(time, day)
        
        # Predict activity
        activity = model.predict(input_data)[0]
        print(f"At {time} on {day}, Mochi will likely be: {activity}")
        
        # Generate and show image
        return generate_mochi_image(activity)
    
    except Exception as e:
        print(f"Prediction failed: {str(e)}")
        return None

# Test it
predict_and_show("15:30", "Saturday")  # Example time/day

In [3]:
from diffusers import StableDiffusionPipeline
import torch

# Try these faster alternatives if sd-v1-5 fails
MODEL_NAME = "runwayml/stable-diffusion-v1-5"  # Official mirror
# MODEL_NAME = "CompVis/stable-diffusion-v1-4"  # Older but reliable

device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    pipe = StableDiffusionPipeline.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,
        safety_checker=None
    ).to(device)
except Exception as e:
    print(f"Error loading model: {e}")
    print("Falling back to CPU with smaller model...")
    pipe = StableDiffusionPipeline.from_pretrained(
        "CompVis/stable-diffusion-v1-4",
        torch_dtype=torch.float32
    ).to("cpu")

# Turbo optimizations
pipe.enable_attention_slicing()
if device == "cuda":
    pipe.unet = torch.compile(pipe.unet)  # 20% speed boost

def generate_mochi_image_fast(activity):
    """Generates images in 2-5 seconds on GPU"""
    image = pipe(
        prompt=f"White Maltese dog {activity.lower()}, realistic",
        negative_prompt="cartoon, blurry, bad anatomy",
        num_inference_steps=15,  # Minimum for decent quality
        guidance_scale=7.0,
    ).images[0]
    
    return image  # Returns PIL Image object

/Users/jasmineho/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jasmineho/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 6/6 [00:08<00:00,  1.49s/it]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly